In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras

In [62]:
df=pd.read_csv('text.csv')

In [63]:
df.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [64]:
df.shape

(416809, 3)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416809 entries, 0 to 416808
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  416809 non-null  int64 
 1   text        416809 non-null  object
 2   label       416809 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 9.5+ MB


In [67]:
df.columns

Index(['Unnamed: 0', 'text', 'label'], dtype='object')

In [72]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [73]:
df.describe()

,label
count,416809.000000
mean,1.554271
std,1.490453
min,0.000000
25%,0.000000
50%,1.000000
75%,3.000000
max,5.000000


In [74]:
df.head()

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [75]:
emotion_dict = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise"
}

In [76]:
# df['length'] = df['text'].apply(len)

In [77]:
print(f"Rating value count: \n{df['label'].value_counts()}")

Rating value count: 
label
1    141067
0    121187
3     57317
4     47712
2     34554
5     14972
Name: count, dtype: int64


In [78]:
print(f"Label value count - percentage distribution: \n{round(df['label'].value_counts()/df.shape[0]*100,2)}")

Label value count - percentage distribution: 
label
1    33.84
0    29.07
3    13.75
4    11.45
2     8.29
5     3.59
Name: count, dtype: float64


In [79]:
data2 = df.groupby('label').apply(lambda x: x.sample(14000)).reset_index(drop=True)

/var/folders/8s/589js3cj3k5gf6dtzp4t6v7r0000gn/T/ipykernel_54843/964725730.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data2 = df.groupby('label').apply(lambda x: x.sample(14000)).reset_index(drop=True)


In [81]:
print(f"Rating value count: \n{data2['label'].value_counts()}")

Rating value count: 
label
0    14000
1    14000
2    14000
3    14000
4    14000
5    14000
Name: count, dtype: int64


In [83]:
data2

,text,label
0,i just feel bad for the workers who have to ha...,0
1,i appreciated her desire to empathize with me ...,0
2,i ly again feeling so depressed,0
3,i feel a little less gloomy about this particu...,0
4,i feel like a rotten apple in a fruit basket,0
...,...,...
83995,i was realised you are engaged in weeks before...,5
83996,i shrugged not feeling particularly enthralled...,5
83997,i wasnt really interested in doing anything to...,5
83998,i still find myself succumbing to daydreams an...,5


In [85]:
X=data2.drop('label',axis=1)

In [90]:
y=data2['label']

In [91]:
X.shape,y.shape

((84000, 1), (84000,))

In [93]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM,Bidirectional
from tensorflow.keras.layers import Dense

In [94]:
voc_size=100000

In [97]:
import nltk
import re
from nltk.corpus import stopwords

In [99]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pranayyb/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [105]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [107]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [108]:
corpus=[]
for i in range(0, len(X)):
    text = re.sub('[^a-zA-Z]', ' ', data2['text'][i])
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    corpus.append(text)

In [109]:
len(corpus)

84000

In [111]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
len(onehot_repr)

84000

In [112]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)

In [113]:
embedded_docs.shape

(84000, 30)

In [114]:
from tensorflow.keras.layers import Dropout

In [139]:
# from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense, GlobalMaxPooling1D

embedding_vector_features = 100
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(500, return_sequences=True)))  # Reduced units for efficiency
model.add(Dropout(0.3))  # Reduced dropout rate
model.add(Bidirectional(LSTM(500, return_sequences=True)))
model.add(Dropout(0.3))  # Reduced dropout rate
model.add(GlobalMaxPooling1D())  # Added pooling layer
model.add(Dense(200, activation='relu'))  # Added dense layer before output
model.add(Dense(6, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [142]:
len(embedded_docs),y.shape

(84000, (84000,))

In [144]:
embedded_docs[83564]

array([31335, 87006, 39902,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0], dtype=int32)

In [145]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [146]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=29)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

Epoch 1/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 1097s 520ms/step - accuracy: 0.6971 - loss: 0.7291 - val_accuracy: 0.9473 - val_loss: 0.1419
Epoch 2/20
2100/2100 ━━━━━━━━━━━━━━━━━━━━ 628s 299ms/step - accuracy: 0.9466 - loss: 0.1307 - val_accuracy: 0.9479 - val_loss: 0.1359
Epoch 3/20
1126/2100 ━━━━━━━━━━━━━━━━━━━━ 4:20 268ms/step - accuracy: 0.9503 - loss: 0.1163

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred[0]

In [41]:
y_pred_classes = np.round(y_pred).astype(int)

In [42]:
y_pred_classes

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0],
       ...,
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [43]:
y_test

array([3, 3, 2, ..., 0, 1, 3])

In [44]:
from sklearn.metrics import accuracy_score

if len(y_pred_classes.shape) > 1 and y_pred_classes.shape[1] > 1:
    y_pred_classes = np.argmax(y_pred_classes, axis=1)

if len(y_test.shape) > 1 and y_test.shape[1] > 1:
    y_test = np.argmax(y_test, axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Accuracy: {accuracy}')


Accuracy: 0.8976190476190476


In [149]:
def predict(text,model):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]
    text = ' '.join(text)
    onehot_repr = [one_hot(text, voc_size)]
    embedded_text = pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
    X_final = np.array(embedded_text)
    y_pred = model.predict(X_final)
    y_pred_classes = np.argmax(y_pred, axis=1)
    emotion=emotion_dict[y_pred_classes[0]]
    return emotion

In [151]:
data="his situation is just making me so angry!"

In [155]:
predict(data,model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


'anger'

In [157]:
import pickle

In [159]:
with open('emotion.pkl', 'wb') as file:
    pickle.dump(model, file)

In [161]:
with open('emotion.pkl', 'rb') as file:
    loaded_model = pickle.load(file)